In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import re
import boto3
import seaborn as sns
import networkx as nx
sns.set(color_codes = True)  #sets nice background color
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Data Extraction

In [ ]:
#AWS Access Key, Secret Access Key and Session Token (This needs to be updated for each run)
key = 'YOUR_KEY_HERE'
secret = 'YOUR_SECRET_HERE'
token = 'YOUR_TOKEN_HERE'

In [ ]:
#Assigning the values of Bucket names and the raw data file name to variables
raw_data_bucket = 'lloydsbanking'
clean_data_bucket = 'lloydscleandata'
raw_data = 'fake_transactional_data.csv'

In [ ]:
#Extracting data from the csv file
lloyds = pd.read_csv(
    f"s3://{raw_data_bucket}/{raw_data}",
    storage_options={
        "key": key,
        "secret": secret,
        "token": token,
    },
)

In [ ]:
#Extracting the csv file
#lloyds = pd.read_csv('/Users/Rahul/OneDrive/Desktop/fake_transactional_data.csv')

In [ ]:
# We have more than 12M points
lloyds.shape

In [ ]:
# Displaying first 5 rows from the data
lloyds.head(5)

In [ ]:
# Checking whether the data types are correct for each column
lloyds.info()

In [ ]:
# Creating a dataframe to count the number of missing values
pd.DataFrame( lloyds.isnull().sum(), columns= ['Number of missing values'])


In [ ]:
# Creating a new variable 'data' and assigning the value of 'Lloyds'
data = lloyds

In [ ]:
# Displaying the number of rows and columns in 'data'
data.shape

# Data Cleaning

In [ ]:
# Filling the missing value in the 'monopoly_money_amount' column with the median value
data['monopoly_money_amount'] = data['monopoly_money_amount'].fillna(data['monopoly_money_amount'].median())

In [ ]:
# Printing 'data'
data

In [ ]:
# Counting the number of missing values in each coulmn of the 'data'
pd.DataFrame( data.isnull().sum(), columns= ['Number of missing values'])

In [ ]:
# Filling the missing values in the 'to_randomly_generated_account' column with the most frequent value
data['to_randomly_generated_account'] = data['to_randomly_generated_account'].fillna(data['to_randomly_generated_account'].mode().iloc[0])

In [ ]:
# 'to_randomly_generated_account' has no missing values
pd.DataFrame( data.isnull().sum(), columns= ['Number of missing values'])

In [ ]:
# Filling the missing values in the 'not_happened_yet_date' column with the most frequent value
data['not_happened_yet_date'] = data['not_happened_yet_date'].fillna(data['not_happened_yet_date'].mode().iloc[0])

In [ ]:
# 'not_happened_yet_date' has no missing values
pd.DataFrame( data.isnull().sum(), columns= ['Number of missing values'])

In [ ]:
# Filling the missing values in the 'from_totally_fake_account' column with the most frequent value
data['from_totally_fake_account'] = data['from_totally_fake_account'].fillna(data['from_totally_fake_account'].mode().iloc[0])

In [ ]:
# Removed all the missing values
pd.DataFrame( data.isnull().sum(), columns= ['Number of missing values'])

In [ ]:
# 'Data' is cleaned and missing values have been filled
data

# Data Preparation

In [ ]:
# Converting the object type of 'from_totally_fake_account' for better understanding
data['from_totally_fake_account'] = data['from_totally_fake_account'].astype('object')

In [ ]:
# Checking whether the data types are correct for each column
data.info()

In [ ]:
# Splitting the 'not_happened_yet_date' into three separate column 'day','month','year'
data[["day", "month", "year"]] = data["not_happened_yet_date"].str.split("/", expand = True)

In [ ]:
# Printing 'data'
data

In [ ]:
# Converting 'not_happened_yet_date' column to datetime data type
data['not_happened_yet_date'] = pd.to_datetime(data['not_happened_yet_date'], format="%d/%m/%Y")

In [ ]:
# Checking whether the data types are correct for each column
data.info()

In [ ]:
# Extracting a column 'day of the week' from 'not_happened_yet_date' column
data["day of the week"] = data["not_happened_yet_date"].dt.day_name()

In [ ]:
# Printing 'data'
data

In [ ]:
# Checking whether the data types are correct for each column
data.info()

In [ ]:
# Defining list of days of the week in a order  
order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

In [ ]:
# Writing data to csv files to load the data to the database
#data.to_csv("/Users/Rahul/OneDrive/Desktop/lloyds_1.csv",index=False)

In [ ]:
# Writing data to csv files to load the data to the database
data.to_csv(
    f"s3://{clean_data_bucket}/lloyds_1.csv",
    index=False,
    storage_options={
        "key": key,
        "secret": secret,
        "token": token,
    },
)

In [ ]:
# Printing data 
data

## Data Visualisation

In [ ]:
# Extracting top 10 most frequently transacting accounts
top_10_acc = data.from_totally_fake_account.value_counts().nlargest(10)

In [ ]:
# Creating a bar plot of the top 10 most common values in the 'from_totally_fake_account' column using seaborn
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.barplot(x=top_10_acc.index, y=top_10_acc.values, palette="Accent_r")
ax.set(xlabel='Account Numbers', ylabel='No. of Transactions')
plt.show()

In [ ]:
# Counting the number of occurrences for every unique values in 'to_randomly_generated_account' using the 'value_counts' method
data.to_randomly_generated_account.value_counts()

In [ ]:
# Finding the top 10 most common values in 'randomly_generated_account' using 'nlargest' to select largest 10 values
top_10_exp = data.to_randomly_generated_account.value_counts().nlargest(10)

In [ ]:
# Printing top_10_exp
top_10_exp

In [ ]:
# Creating a bar plot of the top 10 most common values in the 'to_randomly_generated_account' column using seaborn
sns.set(rc={'figure.figsize':(20,8.27)})
ax = sns.barplot(x=top_10_exp.index, y=top_10_exp.values, palette="Accent_r")
ax.set(xlabel='Popular Expenses', ylabel='No. of Transactions')
plt.show()

## Inference: Customers usually spend the most in Pubs and Bars

# Splitting data into C-C and C-B transactions

In [ ]:
# Creating a dataframe 'df_customer' where 'to_randomly_generated_account' consists of only numeric value
df_customer = data.loc[data['to_randomly_generated_account'].str.isnumeric()]

In [ ]:
# Printing df_customer
df_customer

In [ ]:
# Writing data to csv files to load the data to the database
df_customer.to_csv(
    f"s3://{clean_data_bucket}/split_data/C-CLloyds.csv",
    index=False,
    storage_options={
        "key": key,
        "secret": secret,
        "token": token,
    },
)


In [ ]:
# Creating a dataframe 'df_merch' where index is not present in the 'df-customer' dataframe
df_merch = data.loc[~data.index.isin(df_customer.index)]

In [ ]:
# Displaying the number of rows and columns in 'df_merch'
df_merch.shape

In [ ]:
# Printing 'df_merch'
df_merch

In [ ]:
df_merch.to_csv(
    f"s3://{clean_data_bucket}/split_data/C-BLloyds.csv",
    index=False,
    storage_options={
        "key": key,
        "secret": secret,
        "token": token,
    },
)

## Now we have two seperate dataframes wherein transactions have happened to customers and merchandises

# EDA - Customer - Customer Transactions

In [ ]:
# Displaying the number of rows and columns in 'df_customer'
df_customer.shape

#### There have been 2985833 C-C transactions

In [ ]:
# Displaying first 5 rows from the 'df_customer'
df_customer.head()

In [ ]:
# Finding the average transactions made in C-C transactions
print('Average C-C transactions: ',df_customer.monopoly_money_amount.mean())

# Finding the minimum transaction made in C-C transactions
print('Minimum C-C transactions: ',df_customer.monopoly_money_amount.min())

# Finding the maximum transaction made in C-C transactions
print('Maximum C-C transactions: ',df_customer.monopoly_money_amount.max())

In [ ]:
# Creating a line plot to understand expenditure patterns monthly over the year
sns.lineplot(data=df_customer, x="month", y="monopoly_money_amount")
ax.set_ylabel('Amount')

In [ ]:
# Creating a line plot to understand expenditure patterns daily over the month
sns.lineplot(data=df_customer, x="day", y="monopoly_money_amount")

## Inference : It is evident that by the end of the month, the amount of money transferred is the lowest

In [ ]:
# Creating a line plot to see amount of money transacted by customers over the days of week
sns.lineplot(data=df_customer, x="day of the week", y="monopoly_money_amount")

## Inference: Least amount of money is sent by customers in the weekends

In [ ]:
# Creating a bar chart to show how many transactions occurred in each month for the data in the 'df_customer'
sns.countplot(x=df_customer["month"])

### Most months have almost same number of transactions where it seems to be a slight increase in August

In [ ]:
# Creating a bar chart to show how many transactions occurred on each day of the month for the data in the 'df_customer'
sns.countplot(x=df_customer["day"])

### The bar chart shows that there were fewer transactions made by customers towards the end of the month compared to the beginning of the month.

In [ ]:
# Creating a bar chart to show the number of customer transactions for each day of the week in a order
sns.countplot(x=df_customer["day of the week"], order = order)

### This plot shows that majority of spendings for C-C transactions happen during the weekends. Although the amount spent is less we can see there's a huge differnce between the spendings on weekends and weekdays

# Best cutomer for C-C transactions

In [ ]:
# Finding the best customer from the data 'df_customer' for column 'from_totally_fake_account'
df_best_cust = df_customer.loc[data['from_totally_fake_account'] == 64214]
df_best_cust

## Grouping the customers based on the amount of money transacted to customer (C - C)

In [ ]:
# Customers with transaction amount greater than or equal to 500
df_cust_transact_greater_than_500 = df_customer[df_customer["monopoly_money_amount"] >= 500]

# Customers with transaction amount greater than or equal to 100 and less than 500
df_cust_transact_greater_than_100 = df_customer[(df_customer["monopoly_money_amount"] >= 100) & (df_customer["monopoly_money_amount"] < 500)]

# Customers with transaction amount greater than or equal to 50 and less than 100
df_cust_transact_greater_than_50 = df_customer[(df_customer["monopoly_money_amount"] >= 50) & (df_customer["monopoly_money_amount"] < 100)]

# Customers with transaction amount greater than or equal to 10 and less than 50
df_cust_transact_greater_than_10 = df_customer[(df_customer["monopoly_money_amount"] >= 10) & (df_customer["monopoly_money_amount"] < 50)]

# Customers with transaction amount less than 10
df_cust_transact_lesser_than_10 = df_customer[df_customer["monopoly_money_amount"] < 10]

In [ ]:
# Displaying first 5 rows for 'df_cust_transact_greater_than_500'
df_cust_transact_greater_than_500.head()

In [ ]:
# Displaying first 5 rows for 'df_cust_transact_greater_than_100'
df_cust_transact_greater_than_100.head()

In [ ]:
# Displaying first 5 rows for 'df_cust_transact_greater_than_50'
df_cust_transact_greater_than_50.head()

In [ ]:
# Displaying first 5 rows for 'df_cust_transact_greater_than_10'
df_cust_transact_greater_than_10.head()

In [ ]:
# Displaying first 5 rows for 'df_cust_transact_lesser_than_10'
df_cust_transact_lesser_than_10.head()

In [ ]:
# Displaying the number of rows and columns
print(df_cust_transact_lesser_than_10.shape)
print(df_cust_transact_greater_than_10.shape)
print(df_cust_transact_greater_than_50.shape)
print(df_cust_transact_greater_than_100.shape)
print(df_cust_transact_greater_than_500.shape)

## Grouping the customers based on the frequency of the money spent

In [ ]:
# Importing the mysql.connector to enable connection to a MySQL database
import mysql.connector

In [ ]:
# Establishing connection with MySQL database using provided credantials
from mysql.connector import Error

try:
    connection = mysql.connector.connect(host='lloydsbanking.cu5hh9wuea2p.us-east-1.rds.amazonaws.com',
                                         database='lloyds_banking',
                                         user='admin',
                                         password='latchu1234')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)


In [ ]:
# Retrieving all customer records from Lloyds banking database and converting them into a Pandas dataframe 
cursor.execute("Select * from lloyds_banking.customer")
cus = cursor.fetchall()
customer = pd.DataFrame(cus, columns=[x[0] for x in cursor.description])

In [ ]:
# Retrieving all customer group records from Lloyds banking database and converting them into a Pandas dataframe.
cursor.execute("Select * from lloyds_banking.cus_group")
cus_to_cus = cursor.fetchall()
cus_group = pd.DataFrame(cus_to_cus, columns=[x[0] for x in cursor.description])

In [ ]:
# Retrieving all business group records from Lloyds banking database and converting them into a Pandas dataframe.
cursor.execute("Select * from lloyds_banking.bus_group")
cus_to_bus = cursor.fetchall()
bus_group = pd.DataFrame(cus_to_bus, columns=[x[0] for x in cursor.description])

In [ ]:
# Retrieving all business group monthly records from Lloyds banking database and converting them into a Pandas dataframe.
cursor.execute("Select * from lloyds_banking.bus_group_mon")
bus_to_bus_mon = cursor.fetchall()
bus_group_mon = pd.DataFrame(bus_to_bus_mon, columns=[x[0] for x in cursor.description])

In [ ]:
# Retrieving all customer group monthly records from Lloyds banking database and converting them into a Pandas dataframe.
cursor.execute("Select * from lloyds_banking.cus_group_mon")
cus_to_bus_mon = cursor.fetchall()
cus_group_mon = pd.DataFrame(cus_to_bus_mon, columns=[x[0] for x in cursor.description])

In [ ]:
# Printing 'cus_group'
cus_group

In [ ]:
#Extracting 'to_randomnly_generated_account','Number_of_Transaction' columns from cus_group and storing in cus_group_transaction
cus_group_transaction = cus_group.drop(columns=['to_randomnly_generated_account', 'Number_of_Transaction'])

In [ ]:
# Printing 'cus_group_transaction'
cus_group_transaction

In [ ]:
# Calculating the median of the 'Total_Amount' column in cus_group_transaction
cus_group_transaction.Total_Amount.median()

In [ ]:
# Calculating the maximum of the 'Total_Amount' column in cus_group_transaction
cus_group_transaction.Total_Amount.max()

In [ ]:
# Calculating the minimum of the 'Total_Amount' column in cus_group_transaction
cus_group_transaction.Total_Amount.min()

In [ ]:
# Selecting 'Number_of_Transaction' and 'Total_Amount' columns from the cus_group
col_names =['Number_of_Transaction', 'Total_Amount']
features = cus_group[col_names]
# Standarizing the values and storing the results in scaled_features
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = col_names)



In [ ]:
# Performing KMeans clustering on the standardized cus_group_transaction for different number of clusters
# Calculating the sum of squared errors (SSE) for each cluster
SSE = []
for cluster in range(1,10):
    kmeans = KMeans(n_clusters = cluster, init='k-means++')
    kmeans.fit(scaled_features)
    SSE.append(kmeans.inertia_)
frame = pd.DataFrame({'Cluster':range(1,10), 'SSE':SSE})
# Creating a line graph for plotting the relationship between the number of clusters and SSE
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
# Performing KMeans clustering on the standardized cus_group_transaction using 3 clusters, and storing the result in 'kmeans'
n_clusters = 3
kmeans = KMeans(n_clusters= 3, random_state = 42)
kmeans.fit(scaled_features)

In [ ]:
# Predicting the cluster labels for each row in the standardized cus_group_transaction 
# Adding the result as a new column 'cluster' in a new dataframe called 'frame'
pred = kmeans.predict(scaled_features)
frame = pd.DataFrame(cus_group)
frame['cluster'] = pred

In [ ]:
# Displaying first five rows for 'frame'
frame.head()

In [ ]:
# Displaying the number of occurrences of each cluster label in the 'cluster' column of the 'frame'
frame.cluster.value_counts()

In [ ]:
# Creating a scatter plot for visualzing the clustering results
x_axis = frame['Number_of_Transaction']
y_axis = frame['Total_Amount']
plt.figure(figsize = (10, 8))
sns.scatterplot(x=x_axis, y=y_axis, hue=frame['cluster'])
plt.title('K-means Clustering')
plt.show()


In [ ]:
# Printing 'bus_group'
bus_group

In [ ]:
#Extracting 'to_randomnly_generated_account','Number_of_Transaction' columns from bus_group and storing in bus_group_transaction
bus_group_transaction = bus_group.drop(columns=['to_randomnly_generated_account', 'Number_of_Transaction'])

In [ ]:
# Printing 'bus_group_transaction'
bus_group_transaction

In [ ]:
# Calculating the median of the 'Total_Amount' column in bus_group_transaction
bus_group_transaction.Total_Amount.median()

In [ ]:
# Calculating the maximum of the 'Total_Amount' column in bus_group_transaction
bus_group_transaction.Total_Amount.max()

In [ ]:
# Calculating the minimum of the 'Total_Amount' column in bus_group_transaction
bus_group_transaction.Total_Amount.min()

In [ ]:
# Selecting 'Number_of_Transaction' and 'Total_Amount' columns from the bus_group
col_names = ['Number_of_Transaction', 'Total_Amount']
features2 = bus_group[col_names]
# Standarizing the values and storing the results in scaled_features2
scaler2 = StandardScaler().fit(features2.values)
features2 = scaler.transform(features2.values)
scaled_features2 = pd.DataFrame(features2, columns = col_names)

In [ ]:
# Performing KMeans clustering on the standardized bus_group_transaction for different number of clusters
# Calculating the sum of squared errors (SSE) for each cluster
SSE = []
for cluster in range(1,10):
    kmeans2 = KMeans(n_clusters = cluster, init='k-means++')
    kmeans2.fit(scaled_features2)
    SSE.append(kmeans2.inertia_)
frame = pd.DataFrame({'Cluster':range(1,10), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
# Performing KMeans clustering on the standardized bus_group_transaction using 3 clusters, and storing the result in 'kmeans2'
kmeans2 = KMeans( n_clusters = 3, init='k-means++')
kmeans2.fit(scaled_features2)

In [ ]:
# Predicting the cluster labels for each row in the standardized cus_group_transaction 
# Adding the result as a new column 'cluster' in a new dataframe called 'frame2'
pred2 = kmeans2.predict(scaled_features2)
frame2 = pd.DataFrame(bus_group)
frame2['cluster'] = pred2

In [ ]:
# Printing 'frame2'
frame2

In [ ]:
# Displaying first five rows for 'frame2'
frame2.head()

In [ ]:
# Displaying the number of occurrences of each cluster label in the 'cluster' column of the 'frame2'
frame2.cluster.value_counts()

In [ ]:
# Creating a scatter plot for visualzing the clustering results
x_axis = frame2['Number_of_Transaction']
y_axis = frame2['Total_Amount']
#plt.figure(figsize = (12, 12))
#sns.scatterplot(x=x_axis, y=y_axis, hue=frame2['cluster'])
#sampled_data = frame2.sample(n=10000)
#sns.scatterplot(x=x_axis, y=y_axis, data=sampled_data, hue=sampled_data['cluster'])
plt.figure(figsize = (10, 8))
sns.scatterplot(x=x_axis, y=y_axis, hue=frame2['cluster'])
plt.title('K-means Clustering')
plt.show()

## Overall Customer Segmentation

In [ ]:
# Grouping the 'lloyds' dataframe by 'from_totally_fake_account','to_randomly_generated_account'
# Aggregating the 'monopoly_money_amount' column by sum and the 'from_totally_fake_account' column by count

lloyds_group = lloyds.groupby(['from_totally_fake_account', 'to_randomly_generated_account']).agg({'monopoly_money_amount': 'sum', 'from_totally_fake_account': 'count'})

In [ ]:
# Renaming the columns and resetting the index of the grouped dataframe
lloyds_group.columns = [ 'Total_Amount', 'Number_of_Transactions']
lloyds_group = lloyds_group.reset_index()

In [ ]:
# Printing 'lloyds_group'
lloyds_group

In [ ]:
# Selecting 'Number_of_Transaction' and 'Total_Amount' columns from the lloyds_group
col_names = ['Number_of_Transactions', 'Total_Amount']
features = lloyds_group[col_names]
# new_features = pd.DataFrame(features, columns = col_names)

# Standarizing the values and storing the results in scaled_features
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = col_names)

In [ ]:
# Performing KMeans clustering on the selected standardized features for different number of clusters
# Calculating the sum of squared errors (SSE) for each cluster
SSE = []
for cluster in range(1,15):
    kmeans = KMeans(n_clusters = cluster, init='k-means++')
    kmeans.fit(scaled_features)
    SSE.append(kmeans.inertia_)

# Converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,15), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
# Performing KMeans clustering using 4 clusters, and storing the result in 'kmeans'
kmeans = KMeans( n_clusters = 4, init='k-means++')
kmeans.fit(scaled_features)

In [ ]:
# Predicting the cluster labels for each row and adding the result as a new column 'cluster' in a new dataframe called 'frame'
pred = kmeans.predict(scaled_features)
frame = pd.DataFrame(lloyds_group)
frame['cluster'] = pred

In [ ]:
# Displaying the number of occurrences of each cluster label in the 'cluster' column of the 'frame'
frame.cluster.value_counts()

In [ ]:
# Creating a scatter plot for visualzing the clustering results
plt.figure(figsize = (12, 10))
sns.scatterplot(data=frame, x='Number_of_Transactions', y='Total_Amount', hue=frame['cluster'])
plt.title('Segmentation K-means')
plt.xlim([-100, 500])
plt.ylim([-100, 20000])

plt.show()

In [ ]:
x_axis = frame['Number_of_Transactions']
y_axis = frame['Total_Amount']
plt.figure(figsize = (10, 8))
sns.scatterplot(x=x_axis, y=y_axis, hue=frame['cluster'])
plt.title(' Segmentation K-means')
plt.show()

### Visualising total amount spent by customers in a range of values

In [ ]:
# Creating bins for the 'Total_Amount' column in 'cus_group_transaction'
cus_group_transaction['Amount_bins'] = pd.cut(cus_group_transaction['Total_Amount'], 
                           bins = [0, 10, 25, 50, 75, 100, 200, 500, 1000, 2000, float('inf')], 
                           labels=['0-10', '10-25', '25-50', '50-75', '75-100', '100-200', '200-500', '500-1000', '1000-2000', '2000-'])

# Plotting histogram with Seaborn
ax = sns.histplot(data=cus_group_transaction, x='Amount_bins', discrete=True, stat='count', color='red')
ax.set(xlabel='Total Amount Range', ylabel='No. of Accounts', title= "Histogram for Total Amount of C-C Transactions")
plt.show()

## The majority of customers typically make transactions within the range of 25 to 50 in terms of total amount spent per transaction

In [ ]:
# Obtaining the frequency of transactions of each customer
frequency = cus_group.groupby('from_totally_fake_account')['Number_of_Transaction'].sum()

In [ ]:
# Printing 'frequency'
frequency

In [ ]:
# Reseting the index and converting the series to 'frequency_cus'
frequency_cus = frequency.reset_index()

# Renaming the columns
frequency_cus.columns = ['From_Acc', 'Frequency']

In [ ]:
# Frequency of transaction of each unique customer
frequency_cus

In [ ]:
# Calculating the median of the 'Frequency' column in 'frequency_cus'
frequency_cus.Frequency.median()

In [ ]:
# Calculating the min of the 'Frequency' column in 'frequency_cus'
frequency_cus.Frequency.min()

In [ ]:
# Calculating the max of the 'Frequency' column in 'frequency_cus'
frequency_cus.Frequency.max()

## Visualising frequency of transactions for each unique customer

In [ ]:
# Creating a list of bins for frequency range
bins = [0, 50, 100, 200, 500, 1000, 5000, frequency_cus['Frequency'].max()]

# Creating a new column in the dataframe with frequency range
frequency_cus['Freq_Range'] = pd.cut(frequency_cus['Frequency'], bins)

# Counting the number of accounts in each frequency range
counts = frequency_cus['Freq_Range'].value_counts().sort_index()

# Creating a bar plot showing the number of accounts in each frequency range
plt.bar(counts.index.astype(str), counts.values)
plt.title("Histogram of Frequency of transactions for C-C")
plt.xlabel('Frequency Range')
plt.ylabel('Number of Accounts')
plt.show()

## 1.Most customers send money to other customers about 200-500 times over a year as per data

## 2. Average total amount ~ 70 and Average freq ~ 234, which means that people transfer smaller amount of money in each transaction and have more number of transactions over the year

# Getting a dataframe for Number of unique accounts they have sent money to 

In [ ]:
# Calculating the frequency of transactions made by each customer and store it in new dataframe 'freq'
freq = cus_group.groupby('from_totally_fake_account')['Number_of_Transaction'].count()
freq

In [ ]:
# Calculating the mean frequency of transactions for each customers account
freq.mean()

In [ ]:
# Calculating the median frequency of transactions for each customers account
freq.median()

In [ ]:
# Calculating the maximum frequency of transactions for each customers account
freq.max()

In [ ]:
# Calculating the minimum frequency of transactions for each customers account
freq.min()

In [ ]:
# Reseting the index and converting the series to 'freq_df'
freq_df = freq.reset_index()

# Renaming the columns
freq_df.columns = ['From_Acc', 'To no. of unique accounts']

In [ ]:
# Printing 'freq_df'
freq_df

## Mean and median are very close, so the data is not very skewed

In [ ]:
# Selecting the top 20 customers with the highest frequency of transactions
top_20 = freq.sort_values(ascending=False).head(20)
top_20

In [ ]:
# Assuming the 1st account number is an outlier, let's remove it from the visualisation
top_20 =top_20.tail(-1)

## Top 20 accounts with the hightest unique transactions

In [ ]:
# Creating a bar plot to show top popular expenses made by the customers
ax = sns.barplot( x=top_20.index, y=top_20.values, palette="flare")
ax.set(xlabel='Popular Expenses', ylabel='No. of Transactions')
plt.show()

## Most of the accounts in top 20 range have sent money to around 30 unique accounts

## Grouping cutomers based on the number of unique accounts they have transacted to

In [ ]:
# Selecting customers with frequency greater than or equal to 30
df_cust_freq_greater_than_30 = freq_df[freq_df['To no. of unique accounts'] >= 30]

# Selecting customers with frequency greater than or equal to 25 and less than 30
df_cust_freq_greater_than_25 = freq_df[(freq_df['To no. of unique accounts'] >= 25) & (freq_df['To no. of unique accounts'] < 30)]

# Selecting customers with frequency greater than or equal to 20 and less than 25
df_cust_freq_greater_than_20 = freq_df[(freq_df['To no. of unique accounts'] >= 20) & (freq_df['To no. of unique accounts'] < 25)]

# Selecting customers with frequency greater than or equal to 15 and less than 20
df_cust_freq_greater_than_15 = freq_df[(freq_df['To no. of unique accounts'] >= 15) & (freq_df['To no. of unique accounts'] < 20)]

# Selecting customers with frequency greater than or equal to 10 and less than 15
df_cust_freq_greater_than_10 = freq_df[(freq_df['To no. of unique accounts'] >= 10) & (freq_df['To no. of unique accounts'] < 15)]

# Selecting customers with frequency less than 10
df_cust_freq_lesser_than_10 = freq_df[freq_df['To no. of unique accounts']<10]

In [ ]:
# Printing 'df_cust_freq_greater_than_30'
df_cust_freq_greater_than_30

In [ ]:
# Printing number of customers who have transacted more than 30 times
print("Number of customers who have transacted more than 30 times: ", len(df_cust_freq_greater_than_30))

In [ ]:
# Displaying the first 10 rows of the dataframe
df_cust_freq_greater_than_25.head(10)

In [ ]:
# Printing number of customers who have transacted more than 25 times
print("Number of customers who have transacted more than 25 times: ", len(df_cust_freq_greater_than_25))

In [ ]:
# Displaying the first 10 rows of the dataframe
df_cust_freq_greater_than_20.head(10)

In [ ]:
# Printing number of customers who have transacted more than 20 times
print("Number of customers who have transacted more than 20 times: ", len(df_cust_freq_greater_than_20))

In [ ]:
# Displaying the first 10 rows of the dataframe
df_cust_freq_greater_than_15.head(10)

In [ ]:
# Printing number of customers who have transacted more than 15 times
print("Number of customers who have transacted more than 15 times: ", len(df_cust_freq_greater_than_15))

In [ ]:
# Displaying the first 10 rows of the dataframe
df_cust_freq_greater_than_10.head(10)

In [ ]:
# Printing number of customers who have transacted more than 10 times
print("Number of customers who have transacted more than 10 times: ", len(df_cust_freq_greater_than_10))

In [ ]:
# Displaying the first 10 rows of the dataframe
df_cust_freq_lesser_than_10.head(10)

In [ ]:
# Printing number of customers who have transacted lesser than 10 times
print("Number of customers who have transacted lesser than 10 times: ", len(df_cust_freq_lesser_than_10))

## We observe that the maximum number of unique customer to customer transactions account have transacted between 10 to 20 times in this year

In [ ]:
# Creating a list of bins for frequency range
bins = [0, 10, 15, 20, 25, 30, float('inf')]
labels = ['Less than 10', '10-15', '15-20', '20-25', '25-30', 'More than 30']

# Adding a new column to freq_df with the frequency range labels
freq_df['Range'] = pd.cut(freq_df['To no. of unique accounts'], bins=bins, labels=labels)

# Creating a histogram of the frequency ranges
plt.hist(freq_df['Range'])

# Setting the x-axis labels
plt.xticks(rotation=45)

# Creating a bar plot for frequency of spending
plt.title('Frequency of Spending')
plt.xlabel('Frequency Range')
plt.ylabel('Number of Accounts')
plt.show()

# Visualising unique transactions for top 5 customers as a network

In [ ]:
# Grouping customer transaction by "From_Acc" and calculating the frequency of transactions
freq = cus_group.groupby("from_totally_fake_account").size().reset_index(name="Freq")

# Sorting the data based on frequency and getting the top 5 accounts
top_5 = freq.nlargest(5, "Freq")["from_totally_fake_account"].values.tolist()


In [ ]:
# Removing the first account from the list as it is an outlier
top_5.pop(0)

In [ ]:
# Printing the top 5 accounts
top_5

In [ ]:
# Filtering the customer transaction data to include only transactions from the top 5 accounts
filtered_data = cus_group[cus_group["from_totally_fake_account"].isin(top_5)]

In [ ]:
# Printing the 'filtered_data'
filtered_data

In [ ]:
# Creating a directed graph using networkx
G = nx.DiGraph()

# Adding nodes for each account
G.add_nodes_from(filtered_data["from_totally_fake_account"].unique())
G.add_nodes_from(filtered_data["to_randomnly_generated_account"].unique())

# Adding edges for each transaction
for _, row in filtered_data.iterrows():
    G.add_edge(row["from_totally_fake_account"], row["to_randomnly_generated_account"], weight=row["Number_of_Transaction"])

# Setting up the layout and Creating the graph
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, node_size=500)
nx.draw_networkx_edges(G, pos, alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=10, font_family="sans-serif")

# Adding edge labels
edge_labels = nx.get_edge_attributes(G, "weight")
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

# Removing axis and showing the plot
plt.axis("off")
plt.show()

# Analysis based on customers grouped according to date of transaction

In [ ]:
# Printing 'cus_group_mon'
cus_group_mon

## Visualising top 10 accounts with Maximum transcations in month of January

In [ ]:
# Filtering the data to include only transactions in January
df_jan = cus_group_mon[cus_group_mon["month_"] == 1]

# Grouping the data by 'from_totally_fake_account' and calculating the sum of 'Number_of_Transaction'
df_jan_grouped = df_jan.groupby("from_totally_fake_account")["Number_of_Transaction"].sum()

# Sorting the data in descending order based on the sum of 'Number_of_Transaction'
df_jan_grouped_sorted = df_jan_grouped.sort_values(ascending=False)

# Plotting a bar chart of the top 10 'From Account' with maximum transactions in January
df_jan_grouped_sorted.head(10).plot(kind="bar")
plt.xlabel("From Account")
plt.ylabel("Number of Transactions")
plt.title("Top 10 From Accounts with Maximum Transactions in January")
plt.show()

In [ ]:
# Creating a new DataFrame with the maximum transaction count for each month
max_counts = cus_group_mon.groupby('month_')['Number_of_Transaction'].max()

# Creating a bar chart of the maximum transaction count for each month using seaborn
ax = sns.barplot( x=max_counts.index, y=max_counts.values, palette="cubehelix")
ax.set(xlabel='Month', ylabel='Maximum Number of Transactions')
plt.show()



# Inference: Maximum number of transactions occur during the beginning of the year

# Grouping the data by total number of transactions of each month 

# Visualising the top 10 customers who spend the most number of time in each month

In [ ]:
# Using For loop to loop through each month from Jan to Oct and filtering the data to include only transactions in that month
for month in range(1, 11):
    df_mon = cus_group_mon[cus_group_mon["month_"] == month]

    # Grouping the data by 'from_totally_fake_account' and calculating the sum of 'Number_of_Transaction'
    df_mon_grouped = df_mon.groupby("from_totally_fake_account")["Number_of_Transaction"].sum()

    # Sorting the data in descending order based on the sum of 'Number_of_Transaction'
    df_mon_grouped_sorted = df_mon_grouped.sort_values(ascending=False)

    # Plotting a bar chart of the top 10 'From Account' with maximum transactions in January
    df_mon_grouped_sorted.head(10).plot(kind="bar")
    plt.xlabel("from_totally_fake_account")
    plt.ylabel("Number of Transactions")
    plt.title("Top 10 From Accounts with Maximum Transactions in January")
    plt.show()

In [ ]:
# Grouping the data by 'from_totally_fake_account' and calculating the sum of 'Number_of_Transaction' column for each account
transactions_by_account = cus_group_mon.groupby('from_totally_fake_account').sum()['Number_of_Transaction']

# Finding the top 10 accounts with the maximum number of transactions
top_10_accounts = transactions_by_account.nlargest(10)

# Printing the top 10 accounts excluding the first one
print(top_10_accounts.tail(-1))


In [ ]:
# Grouping the data by 'from_totally_fake_account' and calculating the sum of 'Number_of_Transaction' column for each account
# Sorting the accounts based on the total transaction amount in descending order and getting the top 10 accounts
top_accounts = cus_group_mon.groupby('from_totally_fake_account').sum().sort_values(by='Number_of_Transaction', ascending=False).head(10).index.tolist()

# Filtering the data to include only transactions from the top 10 accounts
top_accounts_df = cus_group_mon[cus_group_mon['from_totally_fake_account'].isin(top_accounts)]

# Using For loop to loop through the top 10 accounts and generate individual line plots for each account's total amount of transaction over the year
for account in top_accounts:
    account_df = top_accounts_df[top_accounts_df['from_totally_fake_account'] == account]

    # Generating individual line plots for each account's total amount of transaction over the year   
    plt.plot(account_df['month_'], account_df['Total_Amount'])
    plt.xlabel('Month')
    plt.ylabel('Total Amount')
    plt.title(f'Total Amount over One Year for Account {account}')
    plt.show()

In [ ]:
# Using different colors for each  customer account
colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k', 'lime', 'purple', 'orange']

# Plotting scatter plot of 'Total Amount' vs 'Number of 'Transaction' for top 10 customers
for i, account in enumerate(top_accounts):
    account_df = top_accounts_df[top_accounts_df['from_totally_fake_account'] == account]
    plt.scatter(account_df['Number_of_Transaction'], account_df['Total_Amount'], c=colors[i], label=f'Account {account}')

# Adding labels and legend to the plot
plt.xlabel('Number of Transactions')
plt.ylabel('Total Amount')
plt.title('Scatter Plot of Number of Transactions vs Total Amount for Top 10 Accounts with Maximum Transactions')
plt.legend()
plt.show()

In [ ]:
# Plotting a histogram for the transaction frequency with 10 bins
plt.hist(cus_group_mon['Number_of_Transaction'], bins=10)
plt.title('Transaction Frequency Histogram')
plt.xlabel('Number of Transactions')
plt.ylabel('Frequency')
plt.show()

# Observation : Most people have transacted less than 15 times in a year

In [ ]:
# Reshaping the data into a pivot table with months as rows, accounts as columns, and Number_of_Transaction as values
pivot = pd.pivot_table(cus_group_mon, values=['Number_of_Transaction'], index='month_', columns='to_randomnly_generated_account')

# Creating the heatmap with Number_of_Transaction as the color scale
sns.heatmap(pivot['Number_of_Transaction'], cmap='YlGnBu')
plt.title('Transaction Frequency Heatmap')
plt.xlabel('Accounts')
plt.ylabel('Month')
plt.show()



# Now lets analyse Customer to Merchandise transactions

In [ ]:
# Displaying the number of rows and columns in 'df_merch'
df_merch.shape

In [ ]:
# Displaying first 5 rows from the 'df_merch'
df_merch.head()

In [ ]:
# Finding the average transactions made in C-B transactions
print('Average C-B transactions: ',df_merch.monopoly_money_amount.mean())

# Finding the minimum transactions made in C-B transactions
print('Minimum C-B transactions: ',df_merch.monopoly_money_amount.min())

# Finding the maximum transactions made in C-B transactions
print('Maximum C-B transactions: ',df_merch.monopoly_money_amount.max())

In [ ]:
# Creating a line plot to understand expenditure patterns monthly over the year
sns.lineplot(data=df_merch, x="month", y="monopoly_money_amount")

In [ ]:
# Creating a line plot to understand expenditure patterns daily over the month
sns.lineplot(data=df_merch, x="day", y="monopoly_money_amount")

In [ ]:
# Creating a bar chart to show how many transactions occurred in each month for the data in the 'df_merch'
sns.countplot(x=df_merch["month"])

In [ ]:
# Creating a bar chart to show how many transactions occurred on each day of the month for the data in the 'df_merch'
sns.countplot(x=df_merch["day"])

### The bar chart shows that there were fewer transactions made by customers towards the end of the month compared to the beginning of the month

In [ ]:
# Creating a bar chart to show the number of customer transactions for each day of the week in a order
sns.countplot(x=df_merch["day of the week"], order=order)
plt.title('Customer spending pattern through the week')


### Unlike the C-C transactions, we don't see a drastic differences between the days of the week for C-B transactions although the most transactions takes place on friday

In [ ]:
# Selecting top 5000 most frequent 'from_totally_fake_account' based on transaction count
df_freq = df_merch['from_totally_fake_account'].value_counts().nlargest(5000)
df_freq

In [ ]:
# Finding the best customer from the data 'df_merch' for column 'from_totally_fake_account'
df_best_cust = df_merch.loc[data['from_totally_fake_account'] == 64214]
df_best_cust

## Grouping the customers based on the amount of money transacted to business (C - B)

In [ ]:
# Customers with transaction amount greater than or equal to 500 
df_merch_transact_greater_than_500 = df_merch[df_merch["monopoly_money_amount"] >= 500]

# Customers with transaction amount greater than or equal to 100 and less than 500
df_merch_transact_greater_than_100 = df_merch[(df_merch["monopoly_money_amount"] >= 100) & (df_merch["monopoly_money_amount"] < 500)]

# Customers with transaction amount greater than or equal to 50 and less than 100
df_merch_transact_greater_than_50 = df_merch[(df_merch["monopoly_money_amount"] >= 50) & (df_merch["monopoly_money_amount"] < 100)]

# Customers with transaction amount greater than or equal to 10 and less than 50
df_merch_transact_greater_than_10 = df_merch[(df_merch["monopoly_money_amount"] >= 10) & (df_merch["monopoly_money_amount"] < 50)]

# Customers with transaction amount less than 10
df_merch_transact_lesser_than_10 = df_merch[df_merch["monopoly_money_amount"] < 10]

In [ ]:
# Displaying first 5 rows for 'df_merch_transact_greater_than_500'
df_merch_transact_greater_than_500.head()

In [ ]:
# Displaying first 5 rows for 'df_merch_transact_greater_than_100'
df_merch_transact_greater_than_100.head()

In [ ]:
# Displaying first 5 rows for 'df_merch_transact_greater_than_50'
df_merch_transact_greater_than_50.head()

In [ ]:
# Displaying first 5 rows for 'df_merch_transact_greater_than_10'
df_merch_transact_greater_than_10.head()

In [ ]:
# Displaying first 5 rows for 'df_merch_transact_lesser_than_10'
df_merch_transact_lesser_than_10.head()

In [ ]:
# Displaying the number of rows and columns
print(df_merch_transact_lesser_than_10.shape)
print(df_merch_transact_greater_than_10.shape)
print(df_merch_transact_greater_than_50.shape)
print(df_merch_transact_greater_than_100.shape)
print(df_merch_transact_greater_than_500.shape)

In [ ]:
# Printing 'bus_group'
bus_group

In [ ]:
#Extracting 'to_randomnly_generated_account','Number_of_Transaction' columns from bus_group and storing in bus_group_transaction
bus_group_transaction = bus_group.drop(columns=['to_randomnly_generated_account', 'Number_of_Transaction'])

In [ ]:
# Printing 'bus_group_transaction'
bus_group_transaction

In [ ]:
# Calculating the median of the 'Total_Amount' column in bus_group_transaction
bus_group_transaction.Total_Amount.median()

In [ ]:
# Calculating the maximum of the 'Total_Amount' column in bus_group_transaction
bus_group_transaction.Total_Amount.max()

In [ ]:
# Calculating the minimum of the 'Total_Amount' column in bus_group_transaction
bus_group_transaction.Total_Amount.min()

In [ ]:
# Creating bins for the 'Total_Amount' column in 'bus_group_transaction'
bus_group_transaction['Amount_bins'] = pd.cut(bus_group_transaction['Total_Amount'], 
                           bins = [0, 10, 25, 50, 75, 100, 200, 500, 1000, 2000, float('inf')], 
                           labels=['0-10', '10-25', '25-50', '50-75', '75-100', '100-200', '200-500', '500-1000', '1000-2000', '2000-'])

# Plotting histogram with seaborn
ax = sns.histplot(data=bus_group_transaction, x='Amount_bins', discrete=True, stat='count', color='red')
ax.set(xlabel='Total Amount Range', ylabel='No. of Accounts', title="Histogram for Total Amount for C-B Transactions")
plt.show()

In [ ]:
# Creating a list of bins
bins = [0, 10, 25, 50, 75, 100, 200, 500, 1000, 2000, float('inf')]

# Using pd.cut() to group the 'Total_Amount' column into the bins
amount_bins = pd.cut(bus_group_transaction['Total_Amount'], bins=bins, labels=['Less than 10', '10-25', '25-50', '50-75', '75-100', '100-200', '200-500', '500-1000', '1000-2000', 'Greater than 2000'])

# Counting the number of accounts in each bin using value_counts()
count = amount_bins.value_counts()

# Creating a bar plot of the counts
count.plot(kind='bar')

# Adding title and labels
plt.title('Histogram of Total Amount for C-B Transactions')
plt.xlabel('Total Amount')
plt.ylabel('Number of Accounts')

# Showing the plot
plt.show()


In [ ]:
# Calculating the total number of transactions per business account and storing it in new dataframe 'Bfrequency'
Bfrequency = bus_group.groupby('from_totally_fake_account')['Number_of_Transaction'].sum()


In [ ]:
# Printing 'Bfrequency'
Bfrequency

In [ ]:
# Resetting the index and converting the series to 'frequency_bus'
frequency_bus = Bfrequency.reset_index()

# Renaming the columns
frequency_bus.columns = ['From_Acc', 'Frequency']

In [ ]:
# Printing 'frequency_bus'
frequency_bus

In [ ]:
# Calculating the mean frequency of transactions for each business account
frequency_bus.Frequency.median()

In [ ]:
# Calculating the minimum frequency of transactions for each business account
frequency_bus.Frequency.min()

In [ ]:
# Calculating the maximum frequency of transactions for each business account
frequency_bus.Frequency.max()

In [ ]:
# Creating a list of bins for frequency range
bins = [0, 50, 100, 200, 500, 1000, 5000, frequency_bus['Frequency'].max()]

# Bin the Frequency column
frequency_bus['Freq_Range'] = pd.cut(frequency_bus['Frequency'], bins)

# Counting the number of accounts in each frequency range
counts = frequency_bus['Freq_Range'].value_counts().sort_index()

# Creating a bar chart for frequency of spending
plt.bar(counts.index.astype(str), counts.values)
plt.title("Histogram of frequency of transactions for C-B")
plt.xlabel('Frequency Range')
plt.ylabel('Number of Accounts')
plt.show()

## Comparing the C-C and C-B transactions

In [ ]:
# Counting the number of rows in two dataframes and storing the count in a list as 'rows'
rows = [len(df_customer.index), len(df_merch.index)]

# Printing 'rows'
rows

In [ ]:
# Defining types of transactions to copmare
types_of_transactions = ["C-C", "C-B"]

In [ ]:
# Creating a bar plot to compare the total number of transaction of C-c and C-B 
fig = plt.figure(figsize=(20, 9))
ax = fig.add_axes([0,0,1,1])
ax.bar(types_of_transactions,rows, width=0.1, color=["blue", "maroon"])
plt.xlabel("Types of transaction")
plt.ylabel("No. of transactions")
plt.title("Comparison of total number of transactions C-C and C-B ")
plt.show()

In [ ]:
# Creating an empty dataframe with index as merchant categories and column as count
df_merch_types = pd.DataFrame(index = ["Cafe", "Pub", "Food", "Supermarket", "Fitness", "Electronics", "Fashion", "Kids", "Books", "Miscellaneous"], columns = ["Count"])

In [ ]:
# Initializing all values in the dataframe to 0
df_merch_types["Count"] = 0

In [ ]:
# Printing the dataframe with all values initialized to 0
df_merch_types

In [ ]:
# Defining a function to categorize merchants based on their names
def count_category(row):
    if re.search(r'RESTAURANT|HOTEL|SANDWICH|KEBAB|TAKEAWAY|LUNCH|COOKSHOP|BUTCHER|STEAK|SEAFOOD|ROASTERIE', row):
        return 'Food'
    elif re.search(r'PUB|BAR|WINE|COCKTAIL|WHISKEY|G&T|LIQUOR|BUTCHER|STEAK|SEAFOOD|ROASTERIE', row):
        return 'Pub'
    elif re.search(r'COFFEE|CAFE|TEA', row):
        return 'Cafe'
    elif re.search(r'SUPERMARKET|DEPARTMENT|GREENGROCER', row):
        return 'Supermarket'
    elif re.search(r'SPORT|GYM|WEIGHTING|TO_BEAN|RUNNING', row):
        return 'Fitness'
    elif re.search(r'ELECTRONIC|TECH', row):
        return 'Electronics'
    elif re.search(r'CLOTH|FASHION', row):
        return 'Fashion'
    elif re.search(r'SCHOOL|CHILDREN|KIDS_ACTIVITY_CENTRE|GAME', row):
        return 'Kids'
    elif re.search(r'BOOK', row):
        return 'Books'
    else:
        return 'Miscellaneous'

# Applying the function to 'to_randomly_generated_account' column of df_merch and counting the categories
df_merch_types["Count"] = df_merch['to_randomly_generated_account'].apply(lambda x: count_category(x)).value_counts()

In [ ]:
# Sorting the dataframe based on the count of merchant categories in descending order
df_merch_types = df_merch_types.sort_values(by=["Count"], ascending=False)

In [ ]:
# Calculating the total number of transactions categorized by merchant types
df_merch_types["Count"].sum()

### Inference : The count of merchant types is same as the number of rows in C-B transactions proving that there was no data loss when grouping different businesses based on their types

In [ ]:
# Creating a line plot to plot the C-B transactions based on their types
sns.lineplot(data=df_merch_types, x=df_merch_types.index, y="Count")

In [ ]:
# Creating a bar plot to plot the C-B transactions based on their types
ax = df_merch_types.plot.bar(rot=0, figsize=(20, 8.27), color=colors)

ax.set_xlabel('Popular Expenses based on categories')
ax.set_ylabel('No. of transactions')
ax.set_title('C-B transactions')

# Showing the plot
plt.show()


In [ ]:
# Setting the size of the figure using seaborn's set function
sns.set(rc={'figure.figsize':(20,8.27)})

# Creating a barplot using seaborn and storing the axes object in variable ax
ax = sns.barplot(x=top_10_exp.index, y=top_10_exp.values, palette="Accent_r")
ax.set(xlabel='Popular Expenses', ylabel='No. of Transactions')
plt.show()

## Inference: The above visualisations shows the difference that, although the most transacted C-B was pub when considering a particular business, the most transacted type of business was cafe when all the businesses were grouped based on their types